In [1]:
import geemap
import ee

In [2]:
# Adjust this to your Google Cloud Project name.
EE_PROJECT_NAME = 'seeing-the-big-picture'

In [3]:
def generate_image(
    region,
    product='COPERNICUS/S2_SR_HARMONIZED',
    min_date='2018-01-01',
    max_date='2020-01-01',
    range_min=0,
    range_max=2000,
    cloud_pct=10
):

    """Generates cloud-filtered, median-aggregated
    Sentinel-2 image from Google Earth Engine using the
    Pythin Earth Engine API.

    Args:
      region (ee.Geometry): The geometry of the area of interest to filter to.
      product (str): Earth Engine asset ID
        You can find the full list of ImageCollection IDs
        at https://developers.google.com/earth-engine/datasets
      min_date (str): Minimum date to acquire collection of satellite images
      max_date (str): Maximum date to acquire collection of satellite images
      range_min (int): Minimum value for visalization range
      range_max (int): Maximum value for visualization range
      cloud_pct (float): The cloud cover percent to filter by (default 10)

    Returns:
      ee.image.Image: Generated Sentinel-2 image clipped to the region of interest
    """

    # Generate median aggregated composite
    image = ee.ImageCollection(product)\
        .filterBounds(region)\
        .filterDate(str(min_date), str(max_date))\
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud_pct))\
        .median()

    # Get RGB bands
    image = image.visualize(bands=['B4', 'B3', 'B2'], min=range_min, max=range_max)
    # Note that the max value of the RGB bands is set to 65535
    # because the bands of Sentinel-2 are 16-bit integers
    # with a full numerical range of [0, 65535] (max is 2^16 - 1);
    # however, the actual values are much smaller than the max value.
    # Source: https://stackoverflow.com/a/63912278/4777141

    return image.clip(region)

In [4]:
center = (14.65087018453776, 27.237812444728334)
img_deg = (0.04494633994145677, 0.04018177619848519)
buffer_factor = 2.0

In [5]:
try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise

# Get the shape geometry for object
center_lon, center_lat = center
img_width_deg, img_height_deg = img_deg
half_width = (img_width_deg * buffer_factor) / 2
half_height = (img_height_deg * buffer_factor) / 2
bbox = [center_lon - half_width, center_lat - half_height, center_lon + half_width, center_lat + half_height]
region = ee.Geometry.Rectangle(bbox)

# Generate RGB image using GEE
image = generate_image(
    region,
    product='COPERNICUS/S2_SR_HARMONIZED', # Sentinel-2A
    min_date='2021-01-01', # Get all images within
    max_date='2021-12-31', # the year 2021
    cloud_pct=10, # Filter out images with cloud cover >= 10.0%
)

# Visualize map
Map = geemap.Map(center=(center_lon, center_lat), zoom=10)
Map

Map(center=[14.65087018453776, 27.237812444728334], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
from ipyleaflet import Map, Marker

m = Map(center=(37.7749, -122.4194), zoom=10)
m

Map(center=[37.7749, -122.4194], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…